# Import Statements

In [1]:
import cv2
import base64
import openai

# Initialize OpenAI Client

In [ ]:
client = openai.OpenAI(api_key="YOUR_OPENAI_API_KEY")

# Split video to frames

In [ ]:
# Load the cricket video
video = cv2.VideoCapture("ex1.mov")

# Calculate video length
length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
fps = video.get(cv2.CAP_PROP_FPS)
video_length_seconds = length / fps

print(f'Video length: {video_length_seconds:.2f} seconds')

# Read frames and encode to base64
base64Frames = []
while video.isOpened():
    success, frame = video.read()
    if not success:
        break
    _, buffer = cv2.imencode(".jpg", frame)
    base64Frames.append(base64.b64encode(buffer).decode("utf-8"))

# Release the video object
video.release()
print(len(base64Frames), "frames read.")


# Ablation test (25 to 100)
 
* We now do Ablation test to see the minimum number of frames that we would need to generate meaningful commmentary for a given event. 
* We find that with a mere 3 to 5 frames per event, we are able to generate meaningful commentary. Hence enabling us to save significant amounts of processing cost.

In [ ]:
sampling_intervals = [25, 30, 40, 50, 80, 100]  # Smaller = more frames sent
results = []

for interval in sampling_intervals:
    print(f"\n>>> Sampling every {interval}th frame")
    
    sampled_frames = base64Frames[0::interval]

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    f"This video shows a single ball being bowled in a cricket match, lasting about {video_length_seconds:.2f} seconds. Write a short, vivid voiceover script (under 30 words) in the style of a cricket commentator. Focus only on the action — describe the delivery, the shot, and the outcome (like a boundary or wicket). Avoid naming players or describing visuals like the crowd or stadium.",
                    *map(lambda x: {"image": x, "resize": 768}, sampled_frames),
                ]
            }
        ],
        max_tokens=100,
    )

    commentary = response.choices[0].message.content
    print(commentary)

    # Optional: Save result
    results.append({
        "interval": interval,
        "frame_count": len(sampled_frames),
        "commentary": commentary
    })